In [3]:
from darts import TimeSeries
from datetime import datetime
from dateutil.parser import parse
from matplotlib.pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import os
import warnings
import itertools
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

%matplotlib inline
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
rcParams['figure.figsize'] = 15, 5

In [60]:
df = pd.read_csv('./public/data/raw_data.csv')

df['TAG'] = df['TAG'] == 'NG'
# df['MELT_TEMP'] = df['MELT_TEMP'].apply(lambda x: x / 10)
# df['MOTORSPEED'] = df['MOTORSPEED'].apply(lambda x: x / 10)
df['STD_DT'] = df['STD_DT'].apply(lambda x : pd.to_datetime(str(x), format='%Y-%m-%d %H:%M:%S'))

df.index = df['NUM']
df.drop(columns='NUM', inplace=True)
df.drop(columns='MELT_WEIGHT', inplace=True) # 이상 유무와 상관관계가 미미함
y = df.pop('TAG')

df

,STD_DT,MELT_TEMP,MOTORSPEED,INSP
NUM,,,,
0,2020-03-04 00:00:00,489,116,3.19
1,2020-03-04 00:00:00,433,78,3.19
2,2020-03-04 00:00:00,464,154,3.19
3,2020-03-04 00:00:00,379,212,3.19
4,2020-03-04 00:00:00,798,1736,3.21
...,...,...,...,...
835195,2020-04-30 23:59:00,755,1743,3.21
835196,2020-04-30 23:59:00,385,206,3.19
835197,2020-04-30 23:59:00,465,148,3.20


In [61]:
df_ = df.iloc[::10].copy(False)

In [62]:
series = TimeSeries.from_dataframe(df_, 'STD_DT', fill_missing_dates=True)

In [63]:
series

<TimeSeries (DataArray) (STD_DT: 83520, component: 3, sample: 1)>
array([[[489.  ],
        [116.  ],
        [  3.19]],

       [[507.  ],
        [128.  ],
        [  3.19]],

       [[474.  ],
        [135.  ],
        [  3.19]],

       ...,

       [[465.  ],
        [108.  ],
        [  3.19]],

       [[498.  ],
        [133.  ],
        [  3.19]],

       [[497.  ],
        [141.  ],
        [  3.2 ]]])
Coordinates:
  * STD_DT     (STD_DT) datetime64[ns] 2020-03-04 ... 2020-04-30T23:59:00
  * component  (component) object 'MELT_TEMP' 'MOTORSPEED' 'INSP'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [64]:
train, val = series[:-20000], series[-20000:-10000]

In [ ]:
from darts.models import TransformerModel, RNNModel

model = TransformerModel(input_chunk_length=60, output_chunk_length=60, batch_size=512 + 128, pl_trainer_kwargs={
      "accelerator": "gpu",
      "devices": [4]
})

model.fit(train, epochs=50, val_series=val)

2022-11-01 14:24:57 pytorch_lightning.utilities.distributed INFO: GPU available: True, used: True
2022-11-01 14:24:57 pytorch_lightning.utilities.distributed INFO: TPU available: False, using: 0 TPU cores
2022-11-01 14:24:57 pytorch_lightning.utilities.distributed INFO: IPU available: False, using: 0 IPUs
2022-11-01 14:24:57 pytorch_lightning.accelerators.gpu INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
2022-11-01 14:24:57 pytorch_lightning.callbacks.model_summary INFO: 
  | Name                | Type                | Params
------------------------------------------------------------
0 | criterion           | MSELoss             | 0     
1 | train_metrics       | MetricCollection    | 0     
2 | val_metrics         | MetricCollection    | 0     
3 | encoder             | Linear              | 256   
4 | positional_encoding | _PositionalEncoding | 0     
5 | transformer         | Transformer         | 548 K 
6 | decoder             | Linear              | 11.7 K
------

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
prediction = model.predict(n=20000)

In [ ]:
sum((series[-10000:] - prediction[10000:])**2)

In [48]:
prediction[10000:10020]

<TimeSeries (DataArray) (STD_DT: 20, component: 4, sample: 1)>
array([[[48.18965209],
        [12.95542413],
        [ 3.1813134 ],
        [ 0.23207526]],

       [[48.17852291],
        [12.9020785 ],
        [ 3.18013451],
        [ 0.2021265 ]],

       [[48.19580675],
        [12.91414706],
        [ 3.19135529],
        [ 0.23863621]],

       [[48.17814659],
        [12.9659948 ],
        [ 3.18419784],
        [ 0.19993451]],

...

       [[48.1683649 ],
        [12.89595247],
        [ 3.19778191],
        [ 0.20259647]],

       [[48.1723369 ],
        [12.90190457],
        [ 3.18293636],
        [ 0.19521221]],

       [[48.1449977 ],
        [12.89126718],
        [ 3.18871263],
        [ 0.20868829]],

       [[48.16682207],
        [12.95270212],
        [ 3.19561204],
        [ 0.2058739 ]]])
Coordinates:
  * STD_DT     (STD_DT) datetime64[ns] 2020-04-24T01:20:00 ... 2020-04-24T01:...
  * component  (component) object 'MELT_TEMP' 'MOTORSPEED' 'INSP' 'TAG'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None